In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2022-10-18 13:26:42--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 2404:6800:4009:81e::200e, 142.250.183.46
Connecting to drive.google.com (drive.google.com)|2404:6800:4009:81e::200e|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tue0lsg2vmidguhl42f7coiot2unfnlk/1666079775000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=b989c992-4d9a-4ebc-84e5-78de178eb624 [following]
--2022-10-18 13:26:43--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tue0lsg2vmidguhl42f7coiot2unfnlk/1666079775000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=b989c992-4d9a-4ebc-84e5-78de178eb624
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 2404:6800:4009:81a::2001, 142.251.42.33
Connecting to doc-08

In [3]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


In [6]:
# Build a basic sentiment network
# Note the embedding layer is first, 
# and the output is only 1 node as it is either 0 or 1 (negative or positive)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9606      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 2s 5ms/step - loss: 0.6920 - accuracy: 0.5210 - val_loss: 0.7053 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6858 - accuracy: 0.5487 - val_loss: 0.6845 - val_accuracy: 0.6115
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6664 - accuracy: 0.6723 - val_loss: 0.6625 - val_accuracy: 0.6917
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6140 - accuracy: 0.7866 - val_loss: 0.6256 - val_accuracy: 0.7118
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.5243 - accuracy: 0.8449 - val_loss: 0.6053 - val_accuracy: 0.6566
Epoch 6/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.8801 - val_loss: 0.5189 - val_accuracy: 0.7594
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.3326 - accuracy: 0.9240 - val_loss: 0.4968 - val_accuracy: 0.7569
Epoch 8/10
50/50 [==

# Get files for visualizing the network

In [8]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(1000, 16)


In [9]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()

In [12]:
vecs = pd.read_csv('vecs.tsv')
vecs

,0.01840447\t-0.06735829\t0.029501734\t-0.0029720976\t0.08185071\t0.093406975\t0.02054429\t0.054682396\t-0.028315207\t-0.16370922\t0.00011075073\t0.06577896\t0.08094932\t-0.030880762\t-0.067492805\t-0.003558829
0,0.030977182\t-0.0006165232\t-0.007123529\t-0.1...
1,0.104726605\t0.10799669\t-0.20571779\t0.035999...
2,0.01909505\t0.03479453\t0.1605823\t0.10104368\...
3,0.0062595396\t-0.027491312\t-0.058808878\t0.00...
4,-0.13257775\t-0.0845717\t-0.04221841\t0.092151...
...,...
993,-0.08097674\t-0.07199546\t0.021019867\t-0.0410...
994,0.11372007\t-0.035691813\t-0.049740203\t0.1121...
995,-0.11176108\t-0.09139739\t0.059777327\t-0.1118...
996,-0.121607445\t-0.1257058\t0.10486209\t0.061322...


In [13]:
meta = pd.read_csv('meta.tsv')
meta

,<OOV>
0,the
1,and
2,i
3,it
4,a
...,...
993,broken
994,effort
995,course
996,breaking


# Predicting Sentiment in New Reviews

In [17]:
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head',
               'nik is good']

print(fake_reviews)

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head', 'nik is good']


In [18]:
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')


HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!



In [19]:
classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
    print(fake_reviews[x])
    print(classes[x])
    print('\n')

I love this phone
[0.9553119]


I hate spaghetti
[0.11617357]


Everything was cold
[0.36956513]


Everything was hot exactly as I wanted
[0.6557263]


Everything was green
[0.5071253]


the host seated us immediately
[0.6559811]


they gave us free chocolate cake
[0.8482616]


not sure about the wilted flowers on the table
[0.05096275]


only works when I stand on tippy toes
[0.8740873]


does not work when I stand on my head
[0.00840619]


nik is good
[0.936667]


